<a href="https://colab.research.google.com/github/hinapyon/Sampling-Frequency-Estimating-of-Human-Activity-Acceleration-Data-using-Transformer-based-Regression/blob/main/my_data%2Bhasc(Transformer-based-Regression-Model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
from decimal import Decimal #小数点桁落ちをなくすためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from scipy import signal  #信号処理のためのモジュール
from scipy.stats import skew, kurtosis  #歪度と尖度を調べるためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn import preprocessing #データを正規化するためのモジュール
from sklearn.preprocessing import StandardScaler  #データを標準化するためのモジュール
from sklearn.preprocessing import LabelEncoder  #カテゴリ変数を数値化するためのモジュール
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score #機械学習モデルの性能評価のためのモジュール
import tensorflow as tf #TensorFlow(Googleが開発したオープンソースの機械学習フレームワーク)
from tensorflow import keras  #TensorFlow用のニューラルネットワークライブラリAPI
from tensorflow.keras import layers #ニューラルネットワークのレイヤーを定義するためのモジュール

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#定数を定義
BINS = 50  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(静止区間が何サンプル連続したら除去するか)
TRAIN_SIZE = 0.8  #機械学習のトレーニングデータの割合
N_ESTIMATORS = 100  #決定木の数

In [4]:
#ディレクトリ内のAMWS020のデータセットのファイル名と周波数を取得する関数
def get_Hz_and_filename(path: str) -> list[int, str]:
    filename = os.listdir(path) #引数のパスのディレクトリの中のファイル名一覧を取得
    Hz_and_filename=[]  #ファイル名と周波数を格納するリストを宣言

    for file in filename:
        Hz = re.search(r'\d+', file)    #正規表現を用いてファイル名の中で一番最初に出てくる数字(周波数)を取得
        if Hz:  #数字の入っていないファイル名があるとエラーを吐くので、このif文でチェックする
            Hz_and_filename.append([int(Hz.group(0)), file])    #ファイル名と周波数を格納

    return Hz_and_filename

In [5]:
#ファイル名と周波数を分けて出力する関数
def divide_Hz_and_filename(Hz_and_filename: list[int, str]) -> tuple[list[int], list[str]]:
    Hz = []
    filename = []
    for row in Hz_and_filename:
      Hz.append(row[0])
      filename.append(row[1])

    return Hz, filename

In [6]:
#自分の加速度データのCSVファイルから3軸加速度を取得する関数
#なんか分割したら3, 4, 5色々おかしくなった
def get_acceleration(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[3]))
            AccY.append(float(row[4]))
            AccZ.append(float(row[5]))

    return AccX, AccY, AccZ

In [7]:
#自分の加速度データのCSVファイルから3軸加速度を取得する関数
def get_accelerationHasc(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[1]))
            AccY.append(float(row[2]))
            AccZ.append(float(row[3]))

    return AccX, AccY, AccZ

In [8]:
#静止区間を除去する関数
def acc_to_remove_stationary_intervals(AccX: list[float], AccY: list[float], AccZ: list[float]) -> list[float]:
    #各軸の加速度の平均を求める
    AvgAccX = sum(AccX) / len(AccX)
    AvgAccY = sum(AccY) / len(AccY)
    AvgAccZ = sum(AccZ) / len(AccZ)

    AvgResultantAcc = math.sqrt(AvgAccX ** 2 + AvgAccY ** 2 + AvgAccZ ** 2) #重力加速度の推定値=合成加速度の平均を求める

    ResultantAcc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(AccX, AccY, AccZ)]   #各時刻の合成加速度を求める

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i = 0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(ResultantAcc):
        if AvgResultantAcc * LOWER_LIMIT < ResultantAcc[i] < AvgResultantAcc * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del ResultantAcc[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書くとstart <= x < stopの範囲が選択される #start番目の値は含まれるがstop番目の値は含まれない
                counter = 0 #カウンターをリセット
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0 #カウンターをリセット
        i += 1

    return ResultantAcc  #静止区間を除去した後のリストを返す

In [9]:
#連続する2サンプルの差分を取る関数
def calculate_differences_of_acceleration(ResultantAcc: list[float]) -> list[float]:
    #DifferenceAcc = [math.fabs(ResultantAcc[i + 1] * 100000 - ResultantAcc[i] * 100000) for i in range(len(ResultantAcc) - 1)]  #100000倍して誤差を取る
    DifferenceAcc = [math.fabs(Decimal(ResultantAcc[i + 1]) - Decimal(ResultantAcc[i])) for i in range(len(ResultantAcc) - 1)]  #連続する2サンプルの誤差を取る
    return DifferenceAcc

In [10]:
#入力された加速度の差分のリストからヒストグラムを作る関数（ビンの範囲が各加速度の差分データの最小値〜最大値）
def create_histogram2(DifferenceAcc_list: list[float]) -> np.histogram:
    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)
    for i in range(len(DifferenceAcc_list)):
        min_value = min(DifferenceAcc_list[i])
        max_value = max(DifferenceAcc_list[i])
        #DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, density=True) #ヒストグラムを作成し、同じ数のビンで区切る
    return DifferenceAcc_hist

In [11]:
#各加速度データをダウンサンプリングする関数
def resampling_Acc(originHz: int, newHz: int, AccX: list[float], AccY: list[float], AccZ: list[float], Hz: np.array) -> tuple[list[float], list[float], list[float], np.array]:
    i = 0   #カウンター変数

    while (Hz[i] == originHz):
        originlen = len(AccX[i])    #元々のデータの長さ
        sampling_factor = int(originlen * (newHz/originHz)) #ダウンサンプリングした後のデータの長さ
        newAccX = signal.resample(AccX[i], sampling_factor)    #データをダウンサンプリング
        newAccY = signal.resample(AccY[i], sampling_factor)    #データをダウンサンプリング
        newAccZ = signal.resample(AccZ[i], sampling_factor)    #データをダウンサンプリング
        AccX.append(newAccX)   #ダウンサンプリングデータを加速度データに追加
        AccY.append(newAccY)   #ダウンサンプリングデータを加速度データに追加
        AccZ.append(newAccZ)   #ダウンサンプリングデータを加速度データに追加
        Hz = np.append(Hz, newHz)   #ダウンサンプリングレートを追加
        i += 1

    return AccX, AccY, AccZ, Hz

In [12]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [13]:
path = "/content/drive/MyDrive/my_walk_data(100Hz15minutes)/"  #Google colab用
#path = "my_walk_data(100Hz15minutesTo20seconds)/"  #local用

In [14]:
#一連の流れ
Hz_and_filename = get_Hz_and_filename(path)
Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
Hz, filename = divide_Hz_and_filename(Hz_and_filename)
Hz = np.array(Hz)

#使う変数を宣言
readAccX, readAccY, readAccZ = [], [], []   #データ読み込む用
AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []

#各データセットからデータを読み込み二次元配列に格納
for i in filename:
    readAccX, readAccY, readAccZ = get_acceleration(path+i)
    AccX.append(readAccX), AccY.append(readAccY), AccZ.append(readAccZ)

#各加速度データをダウンサンプリング
for i in range(9, 1, -1):
    AccX, AccY, AccZ, Hz = resampling_Acc(100, i * 10, AccX, AccY, AccZ, Hz)

#静止区間を除去
for i in range(len(Hz)):
    ResultantAcc.append(acc_to_remove_stationary_intervals(AccX[i], AccY[i], AccZ[i]))

In [15]:
#ヒストグラム作成
for i in range(len(ResultantAcc)):
    DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc[i]))
DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)

In [16]:
pathHasc = "/content/drive/MyDrive/hasc/"
filename = os.listdir(pathHasc) #引数のパスのディレクトリの中のファイル名一覧を取得
#filename.remove(".DS_Store")

In [17]:
#使う変数を宣言
readAccX, readAccY, readAccZ = [], [], []   #データ読み込む用
AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []

#各データセットからデータを読み込み二次元配列に格納
for i in filename:
    readAccX, readAccY, readAccZ = get_accelerationHasc(pathHasc+i)
    AccX.append(readAccX), AccY.append(readAccY), AccZ.append(readAccZ)

HzHasc = np.ones(len(filename)) * 100

#各加速度データをダウンサンプリング
for i in range(9, 1, -1):
    AccX, AccY, AccZ, HzHasc = resampling_Acc(100, i * 10, AccX, AccY, AccZ, HzHasc)

#静止区間を除去
for i in range(len(HzHasc)):
    ResultantAcc.append(acc_to_remove_stationary_intervals(AccX[i], AccY[i], AccZ[i]))

In [18]:
#ヒストグラム作成
for i in range(len(ResultantAcc)):
    DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc[i]))
DifferenceAcc_histHasc = create_histogram2(DifferenceAcc_list)

In [19]:
pathKaggle = "/content/drive/MyDrive/11.Walk/"
filename = os.listdir(pathKaggle) #引数のパスのディレクトリの中のファイル名一覧を取得
#filename.remove(".DS_Store")

In [20]:
#使う変数を宣言
readAccX, readAccY, readAccZ = [], [], []   #データ読み込む用
AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []

#各データセットからデータを読み込み二次元配列に格納
for i in filename:
    readAccX, readAccY, readAccZ = get_accelerationHasc(pathKaggle+i)
    AccX.append(readAccX), AccY.append(readAccY), AccZ.append(readAccZ)

HzKaggle = np.ones(len(filename)) * 100

#各加速度データをダウンサンプリング
for i in range(9, 1, -1):
    AccX, AccY, AccZ, HzKaggle = resampling_Acc(100, i * 10, AccX, AccY, AccZ, HzKaggle)

#静止区間を除去
for i in range(len(HzKaggle)):
    ResultantAcc.append(acc_to_remove_stationary_intervals(AccX[i], AccY[i], AccZ[i]))

In [21]:
#ヒストグラム作成
for i in range(len(ResultantAcc)):
    DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc[i]))
DifferenceAcc_histKaggle = create_histogram2(DifferenceAcc_list)

In [22]:
#標準化
scaler = StandardScaler()
# Fit the scaler to x_train
scaler.fit(DifferenceAcc_hist)
# Use the scaler to transform x_train and x_test
DifferenceAcc_hist = scaler.transform(DifferenceAcc_hist)

In [23]:
#標準化
scaler = StandardScaler()
# Fit the scaler to x_train
scaler.fit(DifferenceAcc_histHasc)
# Use the scaler to transform x_train and x_test
DifferenceAcc_histHasc = scaler.transform(DifferenceAcc_histHasc)

In [24]:
#標準化
scaler = StandardScaler()
# Fit the scaler to x_train
scaler.fit(DifferenceAcc_histKaggle)
# Use the scaler to transform x_train and x_test
DifferenceAcc_histKaggle = scaler.transform(DifferenceAcc_histKaggle)

In [41]:
#transformer regression
#各種パラメータ
NUM_HEADS = 2
KEY_DIM = 25
BINS = 50
DROPOUT = 0.1
N = 1

# Define the input shape
input_shape = (BINS,)
output_shape = (1,)

#形を定義(このモジュールは行列でないとダメっぽい)
inputs_encoder = layers.Input(shape=input_shape)
inputs_decoder = layers.Input(shape=output_shape)

In [42]:
#Encoderに対する入力の形状
x_encoder = layers.Reshape((1, 50))(inputs_encoder)

In [43]:
#Transformer Encoder Layer(BERT)
for i in range(N):
      #Multi-Head-Attention Layer
      attention_encoder = layers.MultiHeadAttention(num_heads=NUM_HEADS, key_dim=KEY_DIM, use_bias=True)(x_encoder, x_encoder, x_encoder)

      #Dropout Layer
      attention_encoder = layers.Dropout(rate=DROPOUT)(attention_encoder)
      #Add & Norm Layer
      attention_encoder = layers.LayerNormalization()(x_encoder + attention_encoder)

      #Feed-Forward-Network
      ffn_encoder = layers.Dense(BINS * 4, use_bias=True, activation="relu")(attention_encoder)
      ffn_encoder = layers.Dense(BINS, use_bias=True)(ffn_encoder)

      #Dropout Layer
      ffn_encoder = layers.Dropout(rate=DROPOUT)(ffn_encoder)
      #Add & Norm Layer
      x_encoder = layers.LayerNormalization()(attention_encoder + ffn_encoder)

In [44]:
x = layers.Flatten()(x_encoder)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="relu")(x)

In [45]:
model = keras.Model(inputs=inputs_encoder, outputs=outputs)

In [46]:
# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.mean_squared_error,
    metrics=[keras.metrics.mean_squared_error],
)

In [55]:
mae = []
mse = []
r2 = []

In [56]:
#自分ので構築→Hascので試す
history = model.fit(DifferenceAcc_hist, Hz, batch_size=32, epochs=50, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_histHasc), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, HzHasc))
mse.append(mean_squared_error(y_pred, HzHasc))
r2.append(r2_score(y_pred, HzHasc))

Epoch 1/50
6/6 [==============================] - 0s 18ms/step - loss: 757.9605 - mean_squared_error: 757.9605 - val_loss: 996.9775 - val_mean_squared_error: 996.9775
Epoch 2/50
6/6 [==============================] - 0s 12ms/step - loss: 272.1977 - mean_squared_error: 272.1977 - val_loss: 383.6170 - val_mean_squared_error: 383.6170
Epoch 3/50
6/6 [==============================] - 0s 14ms/step - loss: 283.5361 - mean_squared_error: 283.5361 - val_loss: 376.9479 - val_mean_squared_error: 376.9479
Epoch 4/50
6/6 [==============================] - 0s 12ms/step - loss: 190.3436 - mean_squared_error: 190.3436 - val_loss: 398.1996 - val_mean_squared_error: 398.1996
Epoch 5/50
6/6 [==============================] - 0s 16ms/step - loss: 164.8629 - mean_squared_error: 164.8629 - val_loss: 439.0816 - val_mean_squared_error: 439.0816
Epoch 6/50
6/6 [==============================] - 0s 14ms/step - loss: 153.1382 - mean_squared_error: 153.1382 - val_loss: 419.9609 - val_mean_squared_error: 419.960

In [49]:
#自分ので構築→Kaggleので試す
history = model.fit(DifferenceAcc_hist, Hz, batch_size=32, epochs=50, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_histKaggle), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, HzKaggle))
mse.append(mean_squared_error(y_pred, HzKaggle))
r2.append(r2_score(y_pred, HzKaggle))

Epoch 1/50
6/6 [==============================] - 0s 40ms/step - loss: 160.4266 - mean_squared_error: 160.4266 - val_loss: 412.0172 - val_mean_squared_error: 412.0172
Epoch 2/50
6/6 [==============================] - 0s 25ms/step - loss: 179.5195 - mean_squared_error: 179.5195 - val_loss: 516.1815 - val_mean_squared_error: 516.1815
Epoch 3/50
6/6 [==============================] - 0s 20ms/step - loss: 165.1607 - mean_squared_error: 165.1607 - val_loss: 525.3163 - val_mean_squared_error: 525.3163
Epoch 4/50
6/6 [==============================] - 0s 25ms/step - loss: 187.1523 - mean_squared_error: 187.1523 - val_loss: 458.3602 - val_mean_squared_error: 458.3602
Epoch 5/50
6/6 [==============================] - 0s 22ms/step - loss: 188.9974 - mean_squared_error: 188.9974 - val_loss: 416.6579 - val_mean_squared_error: 416.6579
Epoch 6/50
6/6 [==============================] - 0s 18ms/step - loss: 156.2565 - mean_squared_error: 156.2565 - val_loss: 485.5429 - val_mean_squared_error: 485.542

In [57]:
#Hascで構築→自分ので試す
history = model.fit(DifferenceAcc_histHasc, HzHasc, batch_size=32, epochs=50, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_hist), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, Hz))
mse.append(mean_squared_error(y_pred, Hz))
r2.append(r2_score(y_pred, Hz))

Epoch 1/50
12/12 [==============================] - 0s 18ms/step - loss: 457.5711 - mean_squared_error: 457.5711 - val_loss: 918.5728 - val_mean_squared_error: 918.5728
Epoch 2/50
12/12 [==============================] - 0s 14ms/step - loss: 323.9331 - mean_squared_error: 323.9331 - val_loss: 713.1569 - val_mean_squared_error: 713.1569
Epoch 3/50
12/12 [==============================] - 0s 13ms/step - loss: 247.6858 - mean_squared_error: 247.6858 - val_loss: 498.2965 - val_mean_squared_error: 498.2965
Epoch 4/50
12/12 [==============================] - 0s 13ms/step - loss: 212.3838 - mean_squared_error: 212.3838 - val_loss: 505.7456 - val_mean_squared_error: 505.7456
Epoch 5/50
12/12 [==============================] - 0s 14ms/step - loss: 166.6045 - mean_squared_error: 166.6045 - val_loss: 456.1385 - val_mean_squared_error: 456.1385
Epoch 6/50
12/12 [==============================] - 0s 14ms/step - loss: 150.6330 - mean_squared_error: 150.6330 - val_loss: 495.0816 - val_mean_squared_er

In [51]:
#Hascで構築→Kaggleので試す
history = model.fit(DifferenceAcc_histHasc, HzHasc, batch_size=32, epochs=50, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_histKaggle), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, HzKaggle))
mse.append(mean_squared_error(y_pred, HzKaggle))
r2.append(r2_score(y_pred, HzKaggle))

Epoch 1/50
12/12 [==============================] - 0s 12ms/step - loss: 154.5461 - mean_squared_error: 154.5461 - val_loss: 521.9749 - val_mean_squared_error: 521.9749
Epoch 2/50
12/12 [==============================] - 0s 10ms/step - loss: 135.0904 - mean_squared_error: 135.0904 - val_loss: 362.1398 - val_mean_squared_error: 362.1398
Epoch 3/50
12/12 [==============================] - 0s 11ms/step - loss: 160.4471 - mean_squared_error: 160.4471 - val_loss: 513.8606 - val_mean_squared_error: 513.8606
Epoch 4/50
12/12 [==============================] - 0s 12ms/step - loss: 143.0761 - mean_squared_error: 143.0761 - val_loss: 476.5786 - val_mean_squared_error: 476.5786
Epoch 5/50
12/12 [==============================] - 0s 10ms/step - loss: 154.2027 - mean_squared_error: 154.2027 - val_loss: 243.7733 - val_mean_squared_error: 243.7733
Epoch 6/50
12/12 [==============================] - 0s 11ms/step - loss: 160.0779 - mean_squared_error: 160.0779 - val_loss: 641.4351 - val_mean_squared_er

In [52]:
#Kaggleで構築→自分ので試す
history = model.fit(DifferenceAcc_histKaggle, HzKaggle, batch_size=32, epochs=50, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_hist), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, Hz))
mse.append(mean_squared_error(y_pred, Hz))
r2.append(r2_score(y_pred, Hz))

Epoch 1/50
43/43 [==============================] - 0s 8ms/step - loss: 328.0284 - mean_squared_error: 328.0284 - val_loss: 394.9089 - val_mean_squared_error: 394.9089
Epoch 2/50
43/43 [==============================] - 0s 7ms/step - loss: 265.7581 - mean_squared_error: 265.7581 - val_loss: 423.2846 - val_mean_squared_error: 423.2846
Epoch 3/50
43/43 [==============================] - 0s 8ms/step - loss: 253.3180 - mean_squared_error: 253.3180 - val_loss: 354.9700 - val_mean_squared_error: 354.9700
Epoch 4/50
43/43 [==============================] - 0s 8ms/step - loss: 240.4245 - mean_squared_error: 240.4245 - val_loss: 432.1332 - val_mean_squared_error: 432.1332
Epoch 5/50
43/43 [==============================] - 0s 8ms/step - loss: 243.9875 - mean_squared_error: 243.9875 - val_loss: 375.7432 - val_mean_squared_error: 375.7432
Epoch 6/50
43/43 [==============================] - 0s 7ms/step - loss: 221.9345 - mean_squared_error: 221.9345 - val_loss: 340.3368 - val_mean_squared_error: 3

In [53]:
#Kaggleので構築→HASCので試す
history = model.fit(DifferenceAcc_histKaggle, HzKaggle, batch_size=32, epochs=50, shuffle=True, validation_split=0.2)
y_pred = tf.cast(tf.clip_by_value(model.predict(DifferenceAcc_histHasc), 20, 100), tf.int32)
mae.append(mean_absolute_error(y_pred, HzHasc))
mse.append(mean_squared_error(y_pred, HzHasc))
r2.append(r2_score(y_pred, HzHasc))

Epoch 1/50
43/43 [==============================] - 1s 12ms/step - loss: 142.4919 - mean_squared_error: 142.4919 - val_loss: 413.3503 - val_mean_squared_error: 413.3503
Epoch 2/50
43/43 [==============================] - 1s 12ms/step - loss: 144.0338 - mean_squared_error: 144.0338 - val_loss: 453.1336 - val_mean_squared_error: 453.1336
Epoch 3/50
43/43 [==============================] - 0s 12ms/step - loss: 141.5326 - mean_squared_error: 141.5326 - val_loss: 430.0413 - val_mean_squared_error: 430.0413
Epoch 4/50
43/43 [==============================] - 0s 11ms/step - loss: 135.1892 - mean_squared_error: 135.1892 - val_loss: 349.0136 - val_mean_squared_error: 349.0136
Epoch 5/50
43/43 [==============================] - 1s 12ms/step - loss: 144.1065 - mean_squared_error: 144.1065 - val_loss: 308.0138 - val_mean_squared_error: 308.0138
Epoch 6/50
43/43 [==============================] - 0s 12ms/step - loss: 132.3889 - mean_squared_error: 132.3889 - val_loss: 417.2016 - val_mean_squared_er

In [58]:
mae

[19.794871794871796, 24.11111111111111]

In [39]:
mse

[704.1752136752136,
 838.1583924349882,
 1301.52,
 393.6010638297872,
 951.0533333333333,
 326.10470085470087]

In [40]:
r2

[-4.426248174553819,
 -3.6160039762601697,
 -2.2387520383073487,
 0.12676362011866837,
 -2.020044339327833,
 0.11111952392531554]